sync linkedin contacts with google contacts?



In [2]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var updateContact, listAllConnections, scrapeEntireLinkedInProfile,
    runSeleniumCell;
var syncLinkedInContacts = () => {
    return importer.interpretAll([
        'update google contact',
        'selenium cell'
    ])
    .then(r => {
        updateContact = r[0].runInNewContext();
        runSeleniumCell = r[1].runInNewContext();
        return runSeleniumCell([
            'scrape linkedin contacts',
            'scrape entire linkedin profile',
        ]);
    })
    .then(r => {
        listAllConnections = r[0];
        scrapeEntireLinkedInProfile = r[1];
        return listAllConnections();
    })
    .then((connections) => importer.runAllPromises(connections.map(c => resolve => {
        var linkedIn;
        return scrapeEntireLinkedInProfile('https://linkedin.com' + c)
        .then(r => {
            linkedIn = r;
            return updateContact({
                displayName: r.name.split(/\s/).join('.*')});
        })
        .then(r => {
            Object.assign(r[0] || {}, linkedIn);
            fs.writeFileSync(
                project + '/linkedin-' + linkedIn.name + '.json',
                JSON.stringify(r, null, 4));
            return resolve(r);
        })
        .catch(e => console.log(e))
    })))
    .catch(e => console.log(e))
};

module.exports = syncLinkedInContacts;
syncLinkedInContacts;



[Function: syncLinkedInContacts]

scrape entire linkedin profile?



In [1]:
var importer = require('../Core');

var getContactInfo = (profile) => {
    return client
    .execute(url => {
        var a = document.createElement('a');
        a.setAttribute('href', url);
        a.href = url;
        a.style.position = 'absolute';
        a.style.zIndex = 4294967295;
        a.style.top = 0;
        a.style.left = 0;
        a.style.bottom = 0;
        a.style.right = 0;
        document.body.appendChild(a);
        setTimeout(() => document.body.removeChild(a), 2000)
    }, profile)
    .click('a[href*="' + profile + '"]')
    .pause(2000)
    .then(() => readLinkedInProfileInfo())
    .catch(e => console.log(e))
}

var loadEntirePage = () => {
    return client
    // load all defered elements by scrolling
    .then(() => getAllUntil(
        false,
        '//*[contains(@class, "pv-deferred-area")]/@id',
        [],
        (a, b) => a == b,
        (i) => i < 30
    ))
    .pause(2000)
    // show more all sections
    .scroll(0, 0)
    .then(() => importer.runAllPromises([
        '//button[contains(., "See more")]',
        '//button[contains(., "more skills")]',
        '//button[contains(., "more education")]',
        '//button[contains(., "more recommendations")]',
    ].map(selector => resolve => {
        return client
        .isVisible(selector)
        .then(is => is
              ? client.click(selector).pause(1000)
              : [])
        .then(e => resolve())
        .catch(e => resolve())
    })))
    .catch(e => console.log(e))
}

var loginLinkedIn, readLinkedInProfileInfo, getAllXPath;
var scrapeEntireLinkedInProfile = (profile) => {
    var contact;
    return client
    .then(() => importer.interpretAll([
        'log in linkedin',
        'scrape linkedin profile',
        'xpath elements'
    ]))
    .then(r => {
        loginLinkedIn = r[0].runInNewContext({client, getCredentials});
        readLinkedInProfileInfo = r[1].runInNewContext({client});
        getAllXPath = r[2].runInNewContext({client});
        return loginLinkedIn();
    })
    .then(() => getContactInfo(profile))
    .then(r => {
        contact = r;
        return loadEntirePage();
    })
    // get extra profile info
    .then(() => {
        return getAllXPath({
            summary: '//p[contains(@class, "section__summary")]//text()',
            experience: [
                '//*[contains(@class, "profile-section__card")]|//*[contains(@class, "profile-section__section-info-item")]',
                ['.//text()']
            ],
            skills: [
                '//*[contains(@class, "skill-entity--featured")]|//*[contains(@class, "skill-entity__pill")]',
                ['.//text()']
            ],
            recommendations: [
                '//*[contains(@class, "recommendation-entity")]',
                ['.//text()']
            ],
            interests: [
                '//*[contains(@class, "interest-entity")]',
                ['.//text()']
            ],
        });
    })
    .then(r => {
        return Object.assign(r, {
            summary: r.summary.join(''),
            experience: r.experience.map(e => typeof e[0] == 'string'
                                         ? e[0]
                                         : e[0].join('')
                                               .trim().split(/\s*\n+\s*/)),
            skills: r.skills.map(e => typeof e[0] == 'string'
                                         ? e[0]
                                         : e[0].join('')
                                               .trim().split(/\s*\n+\s*/)),
            recommendations: r.recommendations.map(e => typeof e[0] == 'string'
                                         ? e[0]
                                         : e[0].join('')
                                               .trim().split(/\s*\n+\s*/)),
            interests: r.interests.map(e => typeof e[0] == 'string'
                                         ? e[0]
                                         : e[0].join('')
                                               .trim().split(/\s*\n+\s*/)),
        }, contact)
    })
    .catch(e => console.log(e))
}
module.exports = scrapeEntireLinkedInProfile;
scrapeEntireLinkedInProfile;



[Function: scrapeEntireLinkedInProfile]

scrape linkedin contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var getAllUntil, getAllXPath;
var listAllConnections = () => {
    var friends = [];
    if(fs.existsSync(project + '/connections.json')) {
        return Promise.resolve(JSON.parse(
            fs.readFileSync(project + '/connections.json')));
    }
    return client
    .then(() => importer.interpretAll([
        'get all elements until',
        'all xpath elements',
    ]))
    .then(r => {
        getAllUntil = r[0].runFresherInNewContext({client});
        getAllXPath = r[1].runFresherInNewContext({client});
        return client.getUrl();
    })
    .url('https://www.linkedin.com/mynetwork/invite-connect/connections/')
    .pause(3000)
    .then(() => getAllUntil(
        false,
        '//a[contains(@href, "/in/")]/@href',
        friends,
        (a, b) => a == b,
        (i) => i < 100
    ))
    .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))
    .then(r => {
        fs.writeFileSync(
            project + '/connections.json',
            JSON.stringify(r, null, 4))
        return r;
    })
    .catch(e => console.log(e))
};
module.exports = listAllConnections;
listAllConnections;



In [ ]:
var importer = require('../Core');

$$.async();
var syncLinkedInContacts;
importer.interpretAll('sync linkedin contacts')
.then(r => syncLinkedInContacts = r.runInNewContext())
.then(() => syncLinkedInContacts())
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))



Initializing webdriver on localhost


sync facebook contacts with google contacts?



update google contact?



In [ ]:
var google = require('googleapis');
var people = google.people('v1');
var importer = require('../Core');

var options = {scopes: [
    'https://www.googleapis.com/auth/contacts'
]};

var findMatch = (arr, contact) => {
    return arr.filter(c => typeof c.names !== 'undefined'
                      && c.names[0].displayName
                          .match(new RegExp(contact.displayName, 'ig')));
}

var googlePromise = (func) => {
    return new Promise((resolve, reject) => func((err, response) => {
        if(err) reject(err);
        try {
        } catch (e) {
            reject(e);
        }
        setTimeout(() => resolve(response), 100);
    })).catch(e => console.log(e));
};

var contactCache = [];
var listContacts = (contact, done = false) => {
    // return matching contacts or empty if there are no more pages
    return googlePromise(
        people.people.connections.list.bind(people.people.connections, {
            resourceName: 'people/me',
            personFields: 'emailAddresses,names,birthdays,phoneNumbers',
            auth: options.auth,
            pageToken: options.pageToken,
        }))
    .then(r => {
        options.pageToken = r.nextPageToken;
        done = !options.pageToken; 
        return r.connections;
    })
    .then(r => {
        for(var c of r) {
            contactCache.push(c);
        }
        if(done) {
            const contacts = findMatch(contactCache, contact);
            return contacts;
        }
        return listContacts(contact, done);
    })
    .catch(e => console.log(e))
};

var updateContact = (contact) => {
    var getOauthClient;
    return importer.interpretAll([
        'import google calendar api',
    ])
    .then(r => {
        getOauthClient = r[0].runInNewContext();
    })
    .then(() => getOauthClient(options))
    .then(() => {
        if(contactCache.length > 0) {
            const contacts = findMatch(contactCache, contact);
            return contacts;
        }
        options.nextPageToken = null;
        contactCache = [];
        return listContacts(contact);
    })
    .catch(e => console.log(e))
};

module.exports = updateContact;
updateContact;

